<a href="https://colab.research.google.com/github/SSPod29/TextMining/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk
!pip install gensim
!pip install pyLDAvis

In [2]:
import nltk
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("punkt")
nltk.download("brown")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [3]:
from nltk.corpus import brown as corpus

In [4]:
for n,item in enumerate(corpus.words(corpus.fileids()[0])[:300]):
    print(item, end=" ")
    if (n%25) ==24:
      print(" ")

The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place .  
The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise  
and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged  
by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan  
Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in  
the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's  
registration and election laws `` are outmoded or inadequate and often ambiguous '' . It recommended that Fulton legislators act `

In [5]:
len(corpus.fileids())

500

In [6]:
docs=[corpus.words(fileid) for fileid in corpus.fileids()]

print(docs[:5])
print("num of docs:", len(docs))

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...], ['Austin', ',', 'Texas', '--', 'Committee', 'approval', ...], ['Several', 'defendants', 'in', 'the', 'Summerdale', ...], ['Oslo', 'The', 'most', 'positive', 'element', 'to', ...], ['East', 'Providence', 'should', 'organize', 'its', ...]]
num of docs: 500


In [37]:
en_stop = nltk.corpus.stopwords.words('english')

en_stop= ["``","/",",.",".,",";","--",":",")","(",'"','&',"'",'),',',"','-','.,','.,"','.-',"?","!",">","<","''"]                  \
         +["0","1","2","3","4","5","6","7","8","9","10","11","12","86","1986","1987","000"]                                                      \
         +["said","say","u","v","mln","ct","net","dlrs","tonne","pct","shr","nil","company","lt","share","year","billion","price", "af"]          \
         +en_stop

In [38]:
from nltk.corpus import wordnet as wn #lemmatize関数のためのimport

def preprocess_word(word, stopwordset):
    
    #1.make words lower  example: Python =>python
    word=word.lower()
    
    #2.remove "," and "."
    if word in [",","."]:
        return None
    
    #3.remove stopword  example: the => (None) 
    if word in stopwordset:
        return None
    
    #4.lemmatize  example: cooked=>cook
    lemma = wn.morphy(word)
    if lemma is None:
        return word

    elif lemma in stopwordset: #lemmatizeしたものがstopwordである可能性がある
        return None
    else:
        return lemma
    

def preprocess_document(document):
    document=[preprocess_word(w, en_stop) for w in document]
    document=[w for w in document if w is not None]
    return document

def preprocess_documents(documents):
    return [preprocess_document(document) for document in documents]

In [40]:
print(preprocess_documents(docs)[0][:25])

['fulton', 'county', 'grand', 'jury', 'friday', 'investigation', "atlanta's", 'recent', 'primary', 'election', 'produce', 'evidence', 'irregularity', 'take', 'place', 'jury', 'term-end', 'presentment', 'city', 'executive', 'committee', 'over-all', 'charge', 'election', 'deserve']


In [41]:
import gensim
from gensim import corpora

In [42]:
dictionary = corpora.Dictionary(preprocess_documents(docs))

corpus_ = [dictionary.doc2bow(doc) for doc in preprocess_documents(docs)]

In [43]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=20,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [44]:
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.005*"would" + 0.004*"one" + 0.003*"make" + 0.003*"time" + 0.003*"go" + 0.003*"come" + 0.003*"may" + 0.003*"know" + 0.002*"get" + 0.002*"even" + 0.002*"two" + 0.002*"also" + 0.002*"take" + 0.002*"could" + 0.002*"like"')
(1, '0.007*"one" + 0.004*"would" + 0.003*"time" + 0.003*"could" + 0.003*"new" + 0.003*"first" + 0.003*"make" + 0.003*"may" + 0.003*"know" + 0.002*"two" + 0.002*"take" + 0.002*"come" + 0.002*"even" + 0.002*"go" + 0.002*"man"')
(2, '0.006*"would" + 0.004*"make" + 0.004*"one" + 0.003*"could" + 0.003*"time" + 0.003*"new" + 0.003*"like" + 0.003*"go" + 0.003*"know" + 0.003*"may" + 0.002*"get" + 0.002*"come" + 0.002*"back" + 0.002*"first" + 0.002*"also"')
(3, '0.005*"make" + 0.004*"one" + 0.003*"would" + 0.003*"could" + 0.003*"time" + 0.003*"man" + 0.002*"take" + 0.002*"come" + 0.002*"see" + 0.002*"use" + 0.002*"may" + 0.002*"new" + 0.002*"go" + 0.002*"know" + 0.002*"two"')
(4, '0.005*"one" + 0.005*"would" + 0.003*"make" + 0.003*"come" + 0.003*"could" + 0.003*"get" + 0.0

In [45]:
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(13, 0.12038134), (15, 0.8631173), (19, 0.015005863)]
document ID 1:[(15, 0.9241518), (18, 0.06266351)]
document ID 2:[(13, 0.9983989)]
document ID 3:[(11, 0.19539519), (13, 0.1903995), (14, 0.2519831), (15, 0.36078298)]
document ID 4:[(1, 0.04034933), (2, 0.06344233), (6, 0.32887703), (8, 0.0127813), (12, 0.31980333), (15, 0.18506171), (17, 0.037417706)]
document ID 5:[(15, 0.39660603), (17, 0.5737659), (19, 0.016871674)]
document ID 6:[(13, 0.7425329), (14, 0.025847645), (15, 0.108774275), (16, 0.11260069)]
document ID 7:[(9, 0.080144525), (13, 0.13712387), (15, 0.78117305)]
document ID 8:[(2, 0.014037522), (13, 0.027191702), (15, 0.8754075), (19, 0.08200236)]
document ID 9:[(15, 0.9914763)]


In [46]:
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [47]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(13, 0.13273722), (15, 0.8534688), (19, 0.012298421)]
['news']
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguo

In [48]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [49]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

In [59]:
print(ldamodel.get_document_topics(corpus_[10]))

[(1, 0.9752794), (19, 0.010674959)]


In [60]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus_,
                                           num_topics=10,
                                           id2word=dictionary,
                                           alpha=0.1,                             #optional LDAのハイパーパラメータalpha
                                           eta=0.1,                                 #optional LDAのハイパーパラメータbeta
                                           #minimum_probability=0.0    #optional 学習結果に格納するトピック・単語の生起確率の下限
                                          )

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [61]:
topics = ldamodel.print_topics(num_words=15)
for topic in topics:
    print(topic)

(0, '0.006*"would" + 0.006*"one" + 0.004*"make" + 0.004*"take" + 0.003*"time" + 0.003*"new" + 0.003*"could" + 0.003*"come" + 0.003*"know" + 0.003*"first" + 0.003*"go" + 0.002*"may" + 0.002*"even" + 0.002*"two" + 0.002*"mr."')
(1, '0.005*"would" + 0.005*"make" + 0.004*"one" + 0.003*"two" + 0.003*"know" + 0.003*"time" + 0.003*"state" + 0.003*"new" + 0.003*"could" + 0.003*"may" + 0.002*"take" + 0.002*"even" + 0.002*"come" + 0.002*"man" + 0.002*"way"')
(2, '0.006*"one" + 0.005*"would" + 0.005*"make" + 0.003*"could" + 0.003*"go" + 0.003*"may" + 0.003*"come" + 0.003*"first" + 0.003*"state" + 0.003*"know" + 0.003*"time" + 0.002*"new" + 0.002*"two" + 0.002*"get" + 0.002*"like"')
(3, '0.008*"one" + 0.004*"would" + 0.004*"make" + 0.003*"time" + 0.003*"go" + 0.003*"could" + 0.003*"man" + 0.003*"first" + 0.003*"new" + 0.003*"like" + 0.003*"may" + 0.003*"come" + 0.003*"back" + 0.002*"get" + 0.002*"take"')
(4, '0.006*"one" + 0.004*"would" + 0.004*"state" + 0.004*"new" + 0.003*"time" + 0.003*"may" + 

In [62]:
for n,item in enumerate(corpus_[:10]):
    print("document ID "+str(n)+":" ,end="")
    print(ldamodel.get_document_topics(item))

document ID 0:[(1, 0.011420488), (5, 0.7165265), (6, 0.26293832)]
document ID 1:[(1, 0.051432535), (8, 0.520248), (9, 0.42200395)]
document ID 2:[(1, 0.75668037), (4, 0.022221135), (6, 0.17397262), (8, 0.032322545)]
document ID 3:[(0, 0.16461389), (4, 0.38226134), (5, 0.15424165), (6, 0.24781263), (9, 0.05062035)]
document ID 4:[(0, 0.7621567), (6, 0.1656654), (8, 0.06773661)]
document ID 5:[(4, 0.51581544), (6, 0.4834624)]
document ID 6:[(0, 0.5386769), (6, 0.22389378), (9, 0.22872317)]
document ID 7:[(4, 0.384729), (5, 0.027721807), (6, 0.5808127)]
document ID 8:[(8, 0.99923384)]
document ID 9:[(6, 0.9877509)]


In [63]:
categories=[corpus.categories(fileid) for fileid in corpus.fileids()]

In [64]:
n=0

#n番目のdocumentのトピック分布
print(ldamodel.get_document_topics(corpus_[n]))

#n番目のdocumentのcategory
print(categories[n])

#n番目のdocumentの生の文章
print(" ".join(docs[n]))

[(4, 0.014157935), (5, 0.7265568), (6, 0.25033936)]
['news']
The Fulton County Grand Jury said Friday an investigation of Atlanta's recent primary election produced `` no evidence '' that any irregularities took place . The jury further said in term-end presentments that the City Executive Committee , which had over-all charge of the election , `` deserves the praise and thanks of the City of Atlanta '' for the manner in which the election was conducted . The September-October term jury had been charged by Fulton Superior Court Judge Durwood Pye to investigate reports of possible `` irregularities '' in the hard-fought primary which was won by Mayor-nominate Ivan Allen Jr. . `` Only a relative handful of such reports was received '' , the jury said , `` considering the widespread interest in the election , the number of voters and the size of this city '' . The jury said it did find that many of Georgia's registration and election laws `` are outmoded or inadequate and often ambiguous 

In [65]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [66]:
lda_display = pyLDAvis.gensim_models.prepare(ldamodel, corpus_, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)